In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark= SparkSession.builder.getOrCreate()

In [4]:
Matches = spark.read.format('csv')\
    .option("header", "True")\
    .option("inferSchema", "True")\
    .load("../data/matches.csv")

In [6]:
Matches.show(3)

+-------+----+----------+-------------------+----------+----+------------+-------------+----------------+----------+-------------+--------------+
| Season|Tour|      Date|               Time|  Opponent|HoAw|ArsenalScore|OpponentScore|         Stadium|Attendance|        Coach|       Referee|
+-------+----+----------+-------------------+----------+----+------------+-------------+----------------+----------+-------------+--------------+
|2017/18|   1|2017-08-11|2024-03-25 20:45:00| Leicester|home|           4|            3|Emirates Stadium|     59387|Arsène Wenger|     Mike Dean|
|2017/18|   2|2017-08-19|2024-03-25 18:30:00|Stoke City|away|           0|            1|  bet365 Stadium|     29459|Arsène Wenger|Andre Marriner|
|2017/18|   3|2017-08-27|2024-03-25 17:00:00| Liverpool|away|           0|            4|         Anfield|     53206|Arsène Wenger|  Craig Pawson|
+-------+----+----------+-------------------+----------+----+------------+-------------+----------------+----------+--------

In [7]:
Matches.schema

StructType([StructField('Season', StringType(), True), StructField('Tour', IntegerType(), True), StructField('Date', DateType(), True), StructField('Time', TimestampType(), True), StructField('Opponent', StringType(), True), StructField('HoAw', StringType(), True), StructField('ArsenalScore', IntegerType(), True), StructField('OpponentScore', IntegerType(), True), StructField('Stadium', StringType(), True), StructField('Attendance', IntegerType(), True), StructField('Coach', StringType(), True), StructField('Referee', StringType(), True)])

In [10]:
Matches.createOrReplaceTempView("Matches")

In [12]:
distinct_matches = spark.sql ("""
select Count(distinct Date) from Matches

""").show()

+--------------------+
|count(DISTINCT Date)|
+--------------------+
|                 214|
+--------------------+



In [13]:
matches = spark.sql ("""
select Count(Date) from Matches

""").show()

+-----------+
|count(Date)|
+-----------+
|        214|
+-----------+



In [14]:
DimMatch= Matches.withColumn("MatchID", monotonically_increasing_id())

In [16]:
DimMatch.show()

+-------+----+----------+-------------------+--------------+----+------------+-------------+-----------------+----------+-------------+---------------+-------+
| Season|Tour|      Date|               Time|      Opponent|HoAw|ArsenalScore|OpponentScore|          Stadium|Attendance|        Coach|        Referee|MatchID|
+-------+----+----------+-------------------+--------------+----+------------+-------------+-----------------+----------+-------------+---------------+-------+
|2017/18|   1|2017-08-11|2024-03-25 20:45:00|     Leicester|home|           4|            3| Emirates Stadium|     59387|Arsène Wenger|      Mike Dean|      0|
|2017/18|   2|2017-08-19|2024-03-25 18:30:00|    Stoke City|away|           0|            1|   bet365 Stadium|     29459|Arsène Wenger| Andre Marriner|      1|
|2017/18|   3|2017-08-27|2024-03-25 17:00:00|     Liverpool|away|           0|            4|          Anfield|     53206|Arsène Wenger|   Craig Pawson|      2|
|2017/18|   4|2017-09-09|2024-03-25 16:0

In [19]:
DimMatch.write.csv('../data/DimMatches.csv', header=True)